In [12]:
import pandas as pd

# 학교의 기본 정보를 가져온다

In [13]:
df1=pd.read_excel('학교개황(20220310기준).xls').loc[:,'학교명':'학교홈페이지']
df1=df1[['학교명']+list(df1.columns[2:5])+['주소','학교홈페이지']]
df1=df1[(df1['학제']=="대학교") |(df1['학제']=="전문대학")]
df1['학교명']=df1['학교명'].apply(lambda x : str(x).split('(')[0])
df1['주소']=df1['주소'].apply(lambda x : str(x).split(' ')[1])
df1=df1[(df1['설립구분']=="국립") |(df1['설립구분']=="사립")]
df1.rename(columns={'지역':'시','주소':'지역'},inplace=True)
data={}
for i in df1['시'].unique():
    data[i]=list(df1[df1['시']==i]['지역'].unique())
df1=df1.groupby('학교명').agg(
    {
        '학제' : 'first',
        '설립구분':'first',
        '시' : ", ".join,
        '지역' : ", ".join,
        '학교홈페이지' : 'first',
    }
).reset_index()
df1['시']=df1['시'].apply(lambda x : ", ".join(list(set(str(x).split(', ')))))
df1['학교홈페이지']=df1['학교홈페이지'].apply(lambda x : str(x).lower())
df1

,학교명,학제,설립구분,시,지역,학교홈페이지
0,가야대학교,대학교,사립,"경북, 경남","김해시, 고령군",www.kaya.ac.kr
1,가천대학교,대학교,사립,경기,성남시,www.gachon.ac.kr
2,가톨릭관동대학교,대학교,사립,강원,강릉시,www.cku.ac.kr
3,가톨릭꽃동네대학교,대학교,사립,충북,청주시,www.kkot.ac.kr
4,가톨릭대학교,대학교,사립,"경기, 서울","부천시, 서초구, 종로구",www.catholic.ac.kr
...,...,...,...,...,...,...
302,호남대학교,대학교,사립,광주,광산구,www.honam.ac.kr
303,호남신학대학교,대학교,사립,광주,남구,www.htus.ac.kr
304,호산대학교,전문대학,사립,경북,경산시,www.hosan.ac.kr
305,호서대학교,대학교,사립,충남,아산시,www.hoseo.ac.kr


# 등록금을 가져온다

In [14]:
df2=pd.read_csv('한국장학재단_대학별 평균등록금_20230428.csv',encoding="EUC-KR").loc[:,'대학명':'평균등록금(원)']
col=['대학명']
if len(df2[df2['평균입학금(원)']!=0])!=0:
    col.append('평균입학금(원)')
col.append('평균등록금(원)')
df2=df2[col]
df2.rename(columns={'대학명':'학교명'},inplace=True)
df2

,학교명,평균등록금(원)
0,강릉원주대학교,4262795
1,강원대학교,4145285
2,경북대학교,4499843
3,경상국립대학교,4042102
4,경인교육대학교,3316000
...,...,...
382,한림성심대학교,5741603
383,한양여자대학교,6189253
384,한영대학교,5611605
385,혜전대학교,5828352


# 장학금을 가져온다

In [15]:
df3=pd.read_csv('한국장학재단_대학별 장학금 수혜 현황_20230831.csv',encoding='EUC-KR')
df3=df3[['학교명','교외장학금 국가', '교외장학금 지방자치단체','교외장학금 사설및기타', '교내장학금 성적우수장학금','교내장학금 근로장학금', '교내장학금 기타장학금']]
df3['학교명']=df3['학교명'].apply(lambda x : str(x).split('(')[0])
group={}
for i in df3.columns[1:]:
    group[i]='first'
df3=df3.groupby('학교명').agg(
    group
).reset_index()
rename={}
for i in df3.columns[1:]:
    rename[i]=i.split(' ')[1]
df3.rename(columns=rename,inplace=True)
df3['교외장학금']=df3['국가']+df3['지방자치단체']+df3['사설및기타']
df3['교내장학금']=df3['성적우수장학금']+df3['근로장학금']+df3['기타장학금']
df3

,학교명,국가,지방자치단체,사설및기타,성적우수장학금,근로장학금,기타장학금,교외장학금,교내장학금
0,ICT폴리텍대학,280892000,0,181420000,33524000,0,0,462312000,33524000
1,가야대학교,4701192873,5438560,46300000,592548990,117497462,474536000,4752931433,1184582452
2,가천대학교,38565866879,97550440,1361531940,6279840000,1445094011,11842199040,40024949259,19567133051
3,가톨릭관동대학교,17562903302,430489570,664507320,2045134790,236506774,3206513498,18657900192,5488155062
4,가톨릭꽃동네대학교,1572115751,2330640,79208000,96244000,125041136,161735634,1653654391,383020770
...,...,...,...,...,...,...,...,...,...
377,호서대학교,28201647866,475156680,289731540,4422236200,1873684248,5366079773,28966536086,11662000221
378,호원대학교,16766637439,38828280,41412000,1181016900,223946596,5158849995,16846877719,6563813491
379,홍익대학교,32583356967,50567180,959136530,17648206300,4354288864,6442353102,33593060677,28444848266
380,화성의과학대학교,72067036,6816400,6701000,105328000,9148564,104653360,85584436,219129924


# 합친다

In [16]:
pd.merge(df1,pd.merge(df2,df3)).to_csv('결과.csv',index=False)